In [1]:
import findspark
findspark.init()
import pyspark
import random
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext(appName='local') 
spark = SparkSession(sc)

In [2]:
#In Spark, create a DataFrame df1 with a column x holding the values 0, …, 19 (hint: spark.range).
#Check the contents of your DataFrame with df1.show()

spark = SparkSession(sc)
df1=spark.range(20).toDF("x")
df1.show(5)

+---+
|  x|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [3]:
#Create a DataFrame df2 with the columns y holding the square root of x and z holding x * x
from pyspark.sql.functions import sqrt, expr, avg, col
df2 = df1.withColumn("y", expr("sqrt(x)")).withColumn("z", expr("x*x"))
df2.show(5)

+---+------------------+---+
|  x|                 y|  z|
+---+------------------+---+
|  0|               0.0|  0|
|  1|               1.0|  1|
|  2|1.4142135623730951|  4|
|  3|1.7320508075688772|  9|
|  4|               2.0| 16|
+---+------------------+---+
only showing top 5 rows



In [6]:
#For df2, find the average of z for the even and odd numbers (hint: “z % 2 = 0”)

df2.select("z").where(expr("z % 2 = 0")).select(avg("z")).toDF("Even").show()
df2.select("z").where(expr("z % 2 != 0")).select(avg("z")).toDF("Odd").show()


df2.select("z").where(expr("z % 2 = 0")).select(avg("z")).toDF("Even").withColumn("z", expr("z % 2 != 0")).select(avg("z")).toDF("Odd").show()

+-----+
| Even|
+-----+
|114.0|
+-----+

+-----+
|  Odd|
+-----+
|133.0|
+-----+



AnalysisException: "cannot resolve '`z`' given input columns: [Even]; line 1 pos 0;\n'Project [Even#142, NOT (('z % 2) = 0) AS z#144]\n+- AnalysisBarrier\n      +- Project [avg(z)#140 AS Even#142]\n         +- Aggregate [avg(z#12L) AS avg(z)#140]\n            +- Filter ((z#12L % cast(2 as bigint)) = cast(0 as bigint))\n               +- Project [z#12L]\n                  +- Project [x#2L, y#9, (x#2L * x#2L) AS z#12L]\n                     +- Project [x#2L, SQRT(cast(x#2L as double)) AS y#9]\n                        +- Project [id#0L AS x#2L]\n                           +- Range (0, 20, step=1, splits=Some(8))\n"

In [5]:
a = col("nosuchcolumn")   #Constructs a column referenced by the name "nosuchcolumn" but not associated to any dataframe
#b = df2["nosuchcolumn"]  #Error - Attempts to retrieve a column that does not exist in dataframe df2
#c = df2.select(a)        #Error - Selecting a column that does not exist in df2
d = df2.select(col("z"))  #Selects only the "z" column from the df2 dataframe
e = df2.select(col("z")).collect() #Returns all the elements from column z, of the dataframe as an array

In [6]:
#Load Batting.csv file
path = "C:/Udvikling/edu/BigData/003/Data/Batting.csv"
battingDF = spark.read.format("com.databricks.spark.csv").load(path, header="true",inferSchema='true')
battingDF.show(10)

+---------+------+-----+------+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+
| playerID|yearID|stint|teamID|lgID|  G| AB|  R|  H| 2B| 3B| HR|RBI| SB| CS| BB| SO| IBB| HBP|  SH|  SF|GIDP|
+---------+------+-----+------+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+
|abercda01|  1871|    1|   TRO|  NA|  1|  4|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|null|null|null|null|null|
| addybo01|  1871|    1|   RC1|  NA| 25|118| 30| 32|  6|  0|  0| 13|  8|  1|  4|  0|null|null|null|null|null|
|allisar01|  1871|    1|   CL1|  NA| 29|137| 28| 40|  4|  5|  0| 19|  3|  1|  2|  5|null|null|null|null|null|
|allisdo01|  1871|    1|   WS3|  NA| 27|133| 28| 44| 10|  2|  2| 27|  1|  1|  0|  2|null|null|null|null|null|
|ansonca01|  1871|    1|   RC1|  NA| 25|120| 29| 39| 11|  3|  0| 16|  6|  2|  2|  1|null|null|null|null|null|
|armstbo01|  1871|    1|   FW1|  NA| 12| 49|  9| 11|  2|  1|  0|  5|  0|  1|  0|  1|null|null|null|null|null|
|barkeal01

In [7]:
#Find the number of runs for each year. 
battingDF.groupBy("yearId").sum("H").alias("runs").orderBy("yearId").show(10)

+------+------+
|yearId|sum(H)|
+------+------+
|  1871|  3101|
|  1872|  4487|
|  1873|  4923|
|  1874|  5226|
|  1875|  6812|
|  1876|  5338|
|  1877|  3705|
|  1878|  3539|
|  1879|  6171|
|  1880|  5946|
+------+------+
only showing top 10 rows



In [8]:
#Find the number of distinct R values in an exact way and in an approximated way with up to 5% error

errorLevel = 0.05
battingRDD = sc.parallelize(battingDF.select("R").collect(), 4)
print("Number of distinct R values in exact way {}".format(battingRDD.count()))
print("Number of distinct R values in approximated way {}".format(battingRDD.countApproxDistinct(errorLevel)))

Number of distinct R values in exact way 102816
Number of distinct R values in approximated way 175


In [105]:
#Find for each year, the maximum number of runs and the player(s) who did the runs.

#batting.select("yearId", "playerID", "H").distinct().groupBy("yearId").max("H").orderBy("yearId").show(300)
#batting.createOrReplaceTempView("battingView")
#spark.sql("select playerId, yearId, max(H) from battingView GROUP BY playerId, yearId, H order by yearId").show()

#battingDF.groupBy("playerId", "yearId").sum("H").orderBy("yearId").show(300)
 
#battingDF.select("playerId", "yearId", max("H")).where("yearId = '1871'").show()

#battingDF.groupBy("playerId", "yearId").sum("H").orderBy("yearId").show()

#battingDF.select("playerId", "yearId", max("H")).rdd.max()[2]

battingDF.createOrReplaceTempView("TEMP_DF")
#spark.sql("SELECT playerId, yearId, H FROM TEMP_DF groupBy playerId").collect()
spark.sql("SELECT playerId, yearId, H FROM TEMP_DF GROUP BY playerId, yearId, H").collect()

[Row(playerId='fleetfr01', yearId=1871, H=2),
 Row(playerId='kesslhe01', yearId=1874, H=17),
 Row(playerId='tippeji01', yearId=1874, H=60),
 Row(playerId='pikeli01', yearId=1875, H=108),
 Row(playerId='thompan01', yearId=1875, H=4),
 Row(playerId='whitede01', yearId=1875, H=136),
 Row(playerId='clappjo01', yearId=1879, H=77),
 Row(playerId='hankifr01', yearId=1881, H=62),
 Row(playerId='wiedmst01', yearId=1881, H=12),
 Row(playerId='harbibi01', yearId=1882, H=23),
 Row(playerId='mcginju01', yearId=1883, H=36),
 Row(playerId='roweda01', yearId=1883, H=80),
 Row(playerId='wheelha01', yearId=1884, H=36),
 Row(playerId='gerhajo01', yearId=1885, H=62),
 Row(playerId='hanlone01', yearId=1885, H=128),
 Row(playerId='mccorji01', yearId=1885, H=23),
 Row(playerId='tenerjo01', yearId=1885, H=0),
 Row(playerId='recciph01', yearId=1886, H=4),
 Row(playerId='zimmech01', yearId=1886, H=3),
 Row(playerId='getzich01', yearId=1887, H=29),
 Row(playerId='irwinjo01', yearId=1887, H=11),
 Row(playerId='ki

In [10]:
#sc.stop()